In [18]:
%%writefile reduction.cu
#include <iostream>                 // Я подключаю ввод-вывод
#include <cuda_runtime.h>           // Я подключаю CUDA

using namespace std;

#define CHECK(call) do {                                                \
    cudaError_t err = (call);                                           \
    if (err != cudaSuccess) {                                           \
        cerr << "CUDA error: " << cudaGetErrorString(err)               \
             << " at line " << __LINE__ << endl;                        \
        return 1;                                                       \
    }                                                                   \
} while(0)

__global__ void reduceSum(const float* input, float* blockSums, int n) {
    __shared__ float sdata[256];              // Я использую shared memory для блока

    int tid = threadIdx.x;                    // Я беру индекс потока в блоке
    int i = blockIdx.x * blockDim.x + tid;    // Я считаю глобальный индекс

    sdata[tid] = (i < n) ? input[i] : 0.0f;   // Я загружаю элемент или 0
    __syncthreads();                          // Я синхронизирую потоки

    for (int s = blockDim.x / 2; s > 0; s >>= 1) { // Я делаю редукцию внутри блока
        if (tid < s) {
            sdata[tid] += sdata[tid + s];     // Я суммирую пары
        }
        __syncthreads();                      // Я синхронизирую после шага
    }

    if (tid == 0) {                           // Первый поток блока
        blockSums[blockIdx.x] = sdata[0];     // Я записываю сумму блока
    }
}

int main() {
    int N = 1024;                              // Я задаю размер массива
    size_t bytes = N * sizeof(float);          // Я считаю байты

    float* h = new float[N];                   // Я создаю массив на CPU
    for (int i = 0; i < N; i++) h[i] = 1.0f;   // Я заполняю единицами

    float *d_in = nullptr, *d_block = nullptr; // Я объявляю указатели GPU
    CHECK(cudaMalloc(&d_in, bytes));           // Я выделяю память под вход
    CHECK(cudaMemcpy(d_in, h, bytes, cudaMemcpyHostToDevice)); // Я копирую вход на GPU

    int threads = 256;                         // Я задаю потоки в блоке
    int blocks  = (N + threads - 1) / threads; // Я считаю блоки

    CHECK(cudaMalloc(&d_block, blocks * sizeof(float))); // Я выделяю память под суммы блоков

    reduceSum<<<blocks, threads>>>(d_in, d_block, N);     // Я запускаю ядро
    CHECK(cudaGetLastError());                            // Я проверяю ошибку запуска
    CHECK(cudaDeviceSynchronize());                       // Я жду завершения ядра

    float* h_block = new float[blocks];                   // Я создаю массив под частичные суммы
    CHECK(cudaMemcpy(h_block, d_block, blocks * sizeof(float), cudaMemcpyDeviceToHost)); // Я копирую суммы блоков

    float sum = 0.0f;                                     // Я считаю финальную сумму на CPU
    for (int i = 0; i < blocks; i++) sum += h_block[i];

    cout << "Reduction result: " << sum << endl;          // Я вывожу результат (должно быть 1024)

    CHECK(cudaFree(d_in));                                // Я освобождаю GPU память
    CHECK(cudaFree(d_block));
    delete[] h;                                           // Я очищаю CPU память
    delete[] h_block;

    return 0;                                             // Я завершаю программу
}


Overwriting reduction.cu


In [21]:
!nvcc reduction.cu -o reduction -gencode arch=compute_75,code=sm_75
!./reduction




Reduction result: 1024


In [26]:
%%writefile scan.cu
#include <iostream>                 // Я подключаю вывод
#include <cuda_runtime.h>           // Я подключаю CUDA runtime

using namespace std;

#define CHECK(call) do {                                                \
    cudaError_t e = (call);                                             \
    if (e != cudaSuccess) {                                             \
        cerr << "CUDA error: " << cudaGetErrorString(e)                 \
             << " at line " << __LINE__ << endl;                        \
        return 1;                                                       \
    }                                                                   \
} while(0)

__global__ void blellochExclusiveScan(const float* input, float* output, int n) {
    __shared__ float temp[256];                 // Я выделяю shared memory на 256 элементов

    int tid = threadIdx.x;                      // Я получаю индекс потока
    int i = tid;                                // Я работаю с одним блоком (0..255)

    temp[tid] = (i < n) ? input[i] : 0.0f;      // Я загружаю вход в shared memory
    __syncthreads();                             // Я синхронизирую потоки

    // -------- UPSWEEP (строю суммы по дереву) --------
    for (int offset = 1; offset < 256; offset <<= 1) {  // Я увеличиваю шаг: 1,2,4,8...
        int idx = (tid + 1) * offset * 2 - 1;           // Я считаю индекс узла дерева
        if (idx < 256) {                                 // Я проверяю границу
            temp[idx] += temp[idx - offset];             // Я складываю левую и правую часть
        }
        __syncthreads();                                 // Я синхронизирую после шага
    }

    // Я обнуляю последний элемент для exclusive scan
    if (tid == 0) temp[255] = 0.0f;                     // Я делаю старт для обратного прохода
    __syncthreads();                                     // Я синхронизирую потоки

    // -------- DOWNSWEEP (распределяю префиксы) --------
    for (int offset = 128; offset > 0; offset >>= 1) {  // Я уменьшаю шаг: 128,64,32...
        int idx = (tid + 1) * offset * 2 - 1;           // Я считаю индекс узла
        if (idx < 256) {                                 // Я проверяю границу
            float t = temp[idx - offset];                // Я сохраняю левое значение
            temp[idx - offset] = temp[idx];              // Я сдвигаю префикс влево
            temp[idx] += t;                              // Я обновляю правый узел
        }
        __syncthreads();                                 // Я синхронизирую после шага
    }

    if (i < n) output[i] = temp[tid];                    // Я записываю результат exclusive scan
}

__global__ void makeInclusive(const float* input, float* scanExclusive, float* scanInclusive, int n) {
    int i = threadIdx.x;                                 // Я беру индекс элемента (0..255)
    if (i < n) scanInclusive[i] = scanExclusive[i] + input[i]; // Я превращаю exclusive в inclusive
}

int main() {
    const int N = 256;                                   // Я беру размер 256 (1 блок, стабильный вариант)
    size_t bytes = N * sizeof(float);                     // Я считаю размер памяти

    float* h_in = new float[N];                           // Я создаю входной массив на CPU
    for (int i = 0; i < N; i++) h_in[i] = 1.0f;           // Я заполняю его единицами

    float *d_in, *d_ex, *d_out;                           // Я объявляю указатели на GPU
    CHECK(cudaMalloc(&d_in, bytes));                      // Я выделяю память под вход
    CHECK(cudaMalloc(&d_ex, bytes));                      // Я выделяю память под exclusive scan
    CHECK(cudaMalloc(&d_out, bytes));                     // Я выделяю память под итоговый inclusive scan

    CHECK(cudaMemcpy(d_in, h_in, bytes, cudaMemcpyHostToDevice)); // Я копирую вход на GPU

    blellochExclusiveScan<<<1, 256>>>(d_in, d_ex, N);     // Я запускаю Blelloch exclusive scan
    CHECK(cudaGetLastError());                            // Я проверяю ошибку запуска
    CHECK(cudaDeviceSynchronize());                       // Я жду завершения

    makeInclusive<<<1, 256>>>(d_in, d_ex, d_out, N);      // Я делаю inclusive scan
    CHECK(cudaGetLastError());                            // Я проверяю ошибку запуска
    CHECK(cudaDeviceSynchronize());                       // Я жду завершения

    float* h_out = new float[N];                          // Я создаю массив под результат на CPU
    CHECK(cudaMemcpy(h_out, d_out, bytes, cudaMemcpyDeviceToHost)); // Я копирую результат

    cout << "Scan result (first 10): ";                   // Я вывожу первые 10 значений
    for (int i = 0; i < 10; i++) cout << h_out[i] << " ";
    cout << endl;

    CHECK(cudaFree(d_in));                                // Я освобождаю GPU память
    CHECK(cudaFree(d_ex));
    CHECK(cudaFree(d_out));
    delete[] h_in;                                        // Я освобождаю CPU память
    delete[] h_out;

    return 0;                                             // Я завершаю программу
}


Overwriting scan.cu


In [28]:
!nvcc scan.cu -o scan -gencode arch=compute_75,code=sm_75
!./scan




Scan result (first 10): 1 2 3 4 5 6 7 8 9 10 


**Вывод**

В данной практической работе я реализовала два параллельных алгоритма на GPU с использованием CUDA: редукцию (суммирование массива) и сканирование (префиксную сумму). Для ускорения я использовала разделяемую память (shared memory), чтобы уменьшить обращения к глобальной памяти. Корректность я проверила на тестовых данных: редукция дала сумму 1024 для массива из единиц, а сканирование должно выдавать префиксные суммы 1,2,3,… для первых элементов.